In [1]:
import os
import copy
import numpy as np
from astropy import constants as const
from astropy.coordinates import SkyCoord, Angle, EarthLocation, Longitude, spherical_to_cartesian, cartesian_to_spherical
from astropy.time import Time, TimeDelta
from astropy import units
from pyuvdata import UVData
import pyuvdata.utils as uvutils
from pyuvsim.data import DATA_PATH

In [2]:
file1 = os.path.join(DATA_PATH, '5km_triangle_1time_1chan.uvfits')

In [3]:
uvd_init = UVData()
uvd_init.read_uvfits(file1)

antenna_diameters is not set. Using known values for HERA.


In [4]:
phase_frame = 'icrs'

In [5]:
print(uvd_init.uvw_array.dtype)
print(uvd_init.antenna_positions.dtype)
# uvd_init.uvw_array = np.float64(uvd1.uvw_array)
uvd_init.unphase_to_drift(phase_frame=phase_frame, use_ant_pos=True)
print(uvd_init.uvw_array.dtype)

float32
float64
float64


In [6]:
uvw_calc = np.zeros_like(uvd_init.uvw_array)
ant_pos_ecef = uvd_init.antenna_positions + uvd_init.telescope_location
ant_pos_enu = uvutils.ENU_from_ECEF(ant_pos_ecef.T, *uvd_init.telescope_location_lat_lon_alt).T
for ind, bl in enumerate(uvd_init.baseline_array):
    ant1 = uvd_init.ant_1_array[ind]
    ant2 = uvd_init.ant_2_array[ind]
    uvw_calc[ind, :] = ant_pos_enu[ant2] - ant_pos_enu[ant1]

print(np.max(np.abs(uvd_init.uvw_array - uvw_calc)))

1.9629996689759532e-13


In [7]:
print(np.min(uvd_init.time_array) - np.max(uvd_init.time_array))

0.0


In [8]:
obs_time = Time(uvd_init.time_array[0], format='jd')
uvd_phase = copy.deepcopy(uvd_init)
uvd_phase.phase_to_time(obs_time, phase_frame=phase_frame)

uvd_phase_unphase = copy.deepcopy(uvd_init)
uvd_phase_unphase.phase_to_time(obs_time, phase_frame=phase_frame)
uvd_phase_unphase.unphase_to_drift(phase_frame=phase_frame)

print(uvd_init.uvw_array)
print(uvd_phase_unphase.uvw_array)
print('')
print(uvd_init.uvw_array - uvd_phase_unphase.uvw_array)
print(np.max(np.abs(uvd_init.uvw_array - uvd_phase_unphase.uvw_array)))

print('')
temp = uvd_init.uvw_array - uvd_phase_unphase.uvw_array
wh_max = np.where(np.abs(temp) == np.max(np.abs(temp)))
print('drift baseline number with max difference:', wh_max)
print('drift baseline vector with max difference:', uvd_init.uvw_array[wh_max[0], :])
print('baseline length for max difference:', np.linalg.norm(uvd_init.uvw_array[wh_max[0], :]))

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-5.00000000e+03  1.05728759e-10  1.77578841e-10]
 [-1.46446610e+03 -3.53553390e+03 -7.79891707e-11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.53553390e+03 -3.53553390e+03 -2.55568011e-10]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]
[[-3.02752942e-05  9.27291148e-06  2.04350268e-05]
 [-5.00000001e+03 -1.93840924e-05 -2.89360930e-05]
 [-1.46446609e+03 -3.53553391e+03 -1.55756218e-05]
 [-3.02752942e-05  9.27291148e-06  2.04350268e-05]
 [ 3.53553389e+03 -3.53553392e+03 -2.31132460e-05]
 [-3.02752942e-05  9.27291148e-06  2.04350268e-05]]

[[ 3.02752942e-05 -9.27291148e-06 -2.04350268e-05]
 [ 6.43268231e-06  1.93841981e-05  2.89362706e-05]
 [-7.36061315e-06  8.70185886e-06  1.55755438e-05]
 [ 3.02752942e-05 -9.27291148e-06 -2.04350268e-05]
 [ 9.70413566e-06  1.68910688e-05  2.31129904e-05]
 [ 3.02752942e-05 -9.27291148e-06 -2.04350268e-05]]
3.0275294180715294e-05

('drift baseline number with max difference:', (array(

In [9]:
uvd_phase2 = copy.deepcopy(uvd_phase_unphase)
uvd_phase2.phase_to_time(obs_time, phase_frame=phase_frame)

uvd_phase_unphase2 = copy.deepcopy(uvd_phase_unphase)
uvd_phase_unphase2.phase_to_time(obs_time, phase_frame=phase_frame)
uvd_phase_unphase2.unphase_to_drift(phase_frame=phase_frame)

print(uvd_init.uvw_array)
print(uvd_phase_unphase2.uvw_array)
print('')
print(uvd_init.uvw_array - uvd_phase_unphase2.uvw_array)
print(np.max(np.abs(uvd_init.uvw_array - uvd_phase_unphase2.uvw_array)))

print('')
temp = uvd_init.uvw_array - uvd_phase_unphase2.uvw_array
wh_max = np.where(np.abs(temp) == np.max(np.abs(temp)))
print('drift baseline number with max difference:', wh_max)
print('drift baseline vector with max difference:', uvd_init.uvw_array[wh_max[0], :])
print('baseline length for max difference:', np.linalg.norm(uvd_init.uvw_array[wh_max[0], :]))

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-5.00000000e+03  1.05728759e-10  1.77578841e-10]
 [-1.46446610e+03 -3.53553390e+03 -7.79891707e-11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.53553390e+03 -3.53553390e+03 -2.55568011e-10]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]
[[-6.05418938e-05  1.04806346e-05  2.61079100e-05]
 [-5.00000001e+03 -1.93840924e-05 -2.89360930e-05]
 [-1.46446613e+03 -3.53553391e+03 -1.51872691e-05]
 [-6.05418938e-05  1.04806346e-05  2.61079100e-05]
 [ 3.53553389e+03 -3.53553392e+03 -2.21389057e-05]
 [-6.05418938e-05  1.04806346e-05  2.61079100e-05]]

[[ 6.05418938e-05 -1.04806346e-05 -2.61079100e-05]
 [ 6.43268231e-06  1.93841981e-05  2.89362706e-05]
 [ 2.51917472e-05  1.28521865e-05  1.51871911e-05]
 [ 6.05418938e-05 -1.04806346e-05 -2.61079100e-05]
 [ 9.70686006e-06  1.85307758e-05  2.21386501e-05]
 [ 6.05418938e-05 -1.04806346e-05 -2.61079100e-05]]
6.0541893808199344e-05

('drift baseline number with max difference:', (array(

In [10]:
zenith_coord = SkyCoord(alt=Angle(90 * units.deg), az=Angle(0 * units.deg),
                        obstime=obs_time, frame='altaz',
                        location=EarthLocation.from_geocentric(uvd_init.telescope_location[0],
                                                               uvd_init.telescope_location[1],
                                                               uvd_init.telescope_location[2],
                                                               unit='m'))
frame_phase_center = zenith_coord.transform_to(phase_frame)

In [11]:
itrs_telescope_location = SkyCoord(x=uvd_init.telescope_location[0] * units.m,
                                   y=uvd_init.telescope_location[1] * units.m,
                                   z=uvd_init.telescope_location[2] * units.m,
                                   representation='cartesian',
                                   frame='itrs', obstime=obs_time)
itrs_lat_lon_alt = uvd_init.telescope_location_lat_lon_alt

frame_telescope_location = itrs_telescope_location.transform_to(phase_frame)

In [12]:
uvw_ecef = uvutils.ECEF_from_ENU(uvd_init.uvw_array.T, *itrs_lat_lon_alt).T
itrs_uvw_coord = SkyCoord(x=uvw_ecef[:, 0] * units.m,
                          y=uvw_ecef[:, 1] * units.m,
                          z=uvw_ecef[:, 2] * units.m,
                          representation='cartesian',
                          frame='itrs', obstime=obs_time)

In [13]:
frame_uvw_coord = itrs_uvw_coord.transform_to(phase_frame)
rt_itrs_uvw_coord = frame_uvw_coord.transform_to('itrs')

In [14]:
print(rt_itrs_uvw_coord.cartesian - itrs_uvw_coord.cartesian)
print(np.max(np.abs(rt_itrs_uvw_coord.cartesian.get_xyz().value - itrs_uvw_coord.cartesian.get_xyz().value)))

print('')
temp = rt_itrs_uvw_coord.cartesian.get_xyz().value - itrs_uvw_coord.cartesian.get_xyz().value
wh_max = np.where(np.abs(temp) == np.max(np.abs(temp)))
print('drift baseline number with max difference:', wh_max)
print('drift baseline vector with max difference:', uvd_init.uvw_array[wh_max[0], :])
print('baseline length for max difference:', np.linalg.norm(uvd_init.uvw_array[wh_max[0], :]))

[( 3.18232924e-05, -2.00339127e-05, -2.46800482e-06),
 (-3.00239772e-05, -1.86937395e-05, -1.88127160e-06),
 (-1.92914158e-05,  3.35974619e-07,  4.76371497e-07),
 ( 3.18232924e-05, -2.00339127e-05, -2.46800482e-06),
 (-2.29831785e-05, -1.94448512e-05, -2.71294266e-06),
 ( 3.18232924e-05, -2.00339127e-05, -2.46800482e-06)] m
3.18232923746109e-05

('drift baseline number with max difference:', (array([0, 0, 0]), array([0, 3, 5])))
('drift baseline vector with max difference:', array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]))
('baseline length for max difference:', 0.0)


In [15]:
frame_rel_uvw = (frame_uvw_coord.cartesian.get_xyz().value.T
                 - frame_telescope_location.cartesian.get_xyz().value)

In [16]:
phased_uvws = uvutils.phase_uvw(frame_phase_center.ra.rad, frame_phase_center.dec.rad, frame_rel_uvw)

In [17]:
print(uvd_phase.uvw_array - phased_uvws)
print(np.max(np.abs(uvd_phase.uvw_array - phased_uvws)))

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
0.0


In [18]:
uvw_rel_positions = uvutils.unphase_uvw(frame_phase_center.ra.rad,
                                        frame_phase_center.dec.rad,
                                        phased_uvws)
rephase_uvws = uvutils.phase_uvw(frame_phase_center.ra.rad, frame_phase_center.dec.rad,
                                 uvw_rel_positions)
print(rephase_uvws - phased_uvws)
print(np.max(np.abs(rephase_uvws - phased_uvws)))

[[0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.95399252e-13 1.78523862e-13]
 [2.27373675e-13 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]]
2.2737367544323206e-13


In [19]:
frame_telescope_location.representation = 'cartesian'
new_frame_uvw_coord = SkyCoord(x=uvw_rel_positions[:, 0] * units.m + frame_telescope_location.x,
                               y=uvw_rel_positions[:, 1] * units.m + frame_telescope_location.y,
                               z=uvw_rel_positions[:, 2] * units.m + frame_telescope_location.z,
                               representation='cartesian',
                               frame=phase_frame, obstime=obs_time)
print(new_frame_uvw_coord.cartesian - frame_uvw_coord.cartesian)
print(np.max(np.abs(new_frame_uvw_coord.cartesian.get_xyz().value - frame_uvw_coord.cartesian.get_xyz().value)))

[(0., 0., 0.), (0., 0., 0.), (0., 0., 0.), (0., 0., 0.), (0., 0., 0.),
 (0., 0., 0.)] m
0.0


In [20]:
new_itrs_uvw_coord = frame_uvw_coord.transform_to('itrs')
print(new_itrs_uvw_coord.cartesian - itrs_uvw_coord.cartesian)
print(np.max(np.abs(new_itrs_uvw_coord.cartesian.get_xyz().value - itrs_uvw_coord.cartesian.get_xyz().value)))

print('')
temp = new_itrs_uvw_coord.cartesian.get_xyz().value - itrs_uvw_coord.cartesian.get_xyz().value
wh_max = np.where(np.abs(temp) == np.max(np.abs(temp)))
print('drift baseline number with max difference:', wh_max)
print('drift baseline vector with max difference:', uvd_init.uvw_array[wh_max[0], :])
print('baseline length for max difference:', np.linalg.norm(uvd_init.uvw_array[wh_max[0], :]))

[( 3.18232924e-05, -2.00339127e-05, -2.46800482e-06),
 (-3.00239772e-05, -1.86937395e-05, -1.88127160e-06),
 (-1.92914158e-05,  3.35974619e-07,  4.76371497e-07),
 ( 3.18232924e-05, -2.00339127e-05, -2.46800482e-06),
 (-2.29831785e-05, -1.94448512e-05, -2.71294266e-06),
 ( 3.18232924e-05, -2.00339127e-05, -2.46800482e-06)] m
3.18232923746109e-05

('drift baseline number with max difference:', (array([0, 0, 0]), array([0, 3, 5])))
('drift baseline vector with max difference:', array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]))
('baseline length for max difference:', 0.0)


In [21]:
drift_uvws = uvutils.ENU_from_ECEF(new_itrs_uvw_coord.cartesian.get_xyz().value,
                                   *itrs_lat_lon_alt).T
print(uvd_init.uvw_array - drift_uvws)
print(np.max(np.abs(uvd_init.uvw_array - drift_uvws)))

print('')
temp = uvd_init.uvw_array - drift_uvws
wh_max = np.where(np.abs(temp) == np.max(np.abs(temp)))
print('drift baseline number with max difference:', wh_max)
print('drift baseline vector with max difference:', uvd_init.uvw_array[wh_max[0], :])
print('baseline length for max difference:', np.linalg.norm(uvd_init.uvw_array[wh_max[0], :]))

[[ 3.02752942e-05 -9.27291148e-06 -2.04350268e-05]
 [ 6.43268231e-06  1.93841981e-05  2.89362706e-05]
 [-7.36061315e-06  8.70185886e-06  1.55755438e-05]
 [ 3.02752942e-05 -9.27291148e-06 -2.04350268e-05]
 [ 9.70413566e-06  1.68910688e-05  2.31129904e-05]
 [ 3.02752942e-05 -9.27291148e-06 -2.04350268e-05]]
3.0275294180715294e-05

('drift baseline number with max difference:', (array([0, 3, 5]), array([0, 0, 0])))
('drift baseline vector with max difference:', array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]))
('baseline length for max difference:', 0.0)
